In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline   

# Printing config:
pd.options.display.float_format = '{:,.4f}'.format
#np.random.seed(42)


# # # # Main Parameters # # # #
dataFilePath = "./usage_records.csv"
#actPlotsFlag = True


# Load the data
data_df = pd.read_csv(dataFilePath)

# Convert relevant columns to datetime type:
date_cols = ['subscription_date', 'redshifted_at']
for col in date_cols:
    data_df.loc[:, col] = pd.to_datetime(data_df.loc[:, col])

In [3]:
# # # # DADA ANALYSIS # # # #

print("\nData head:\n" + str(data_df.head()))
print("\nData shape: " + str(data_df.shape))  

# Checking cols data type and existence of missing values 
print(("\nData info: \n%s") % data_df.info())  
print(("\nMissing values: \n%s") % data_df.isnull().sum( ))  # no missing values

# Check data validity:
groupby_cols = ['id','renewed']
print("\nValidatiov: each 'id' has only one label - %s" % 
      (len(data_df.groupby(groupby_cols).size()) == len(data_df.id.unique())))
groupby_cols = ['id','subscription_date']
print("Validatiov: each 'id' has only one subscription_date - %s" % 
      (len(data_df.groupby(groupby_cols).size()) == len(data_df.id.unique())))
print("Validatiov: all 'redshifted_at' values are later (or equal to) 'subscription_date' - %s" % 
      (sum(data_df.subscription_date <= data_df.redshifted_at) == data_df.shape[0]))
      
# Basic statistics:
num_of_users = len(data_df.id.unique())
num_of_churned_users = len(data_df[data_df['renewed'] == False].id.unique())
print(("\nThere are %d churned users out of %d users (%.4f percent)") % 
      (num_of_churned_users,
       num_of_users,
       100 * num_of_churned_users / num_of_users))
print(("\nPopular features normalized frequency: \n%s") % 
      data_df.feature_name.value_counts(normalize=True).head())

# Analysis per label:
# Number of actions:
groupby_cols = ['renewed', 'id']
label_id_counts_df = data_df.groupby(groupby_cols).size().reset_index(name='counts')
print("\n\nActivity stats of churned users: \n%s" % 
label_id_counts_df[label_id_counts_df['renewed'] == False].counts.describe())
print("\nActivity stats of renewed users: \n%s" % 
label_id_counts_df[label_id_counts_df['renewed'] == True].counts.describe())
# Number of features being used:
unique_fe_by_label_id_df = data_df.groupby(groupby_cols).agg({"feature_name": lambda x: x.nunique()})
unique_fe_by_label_id_df.reset_index(inplace=True)
print("\n\nAmount of used features stats of churned users: \n%s" % 
      unique_fe_by_label_id_df[unique_fe_by_label_id_df['renewed'] == False].feature_name.describe())
print("\nAmount of used features stats of renewed users: \n%s" % 
      unique_fe_by_label_id_df[unique_fe_by_label_id_df['renewed'] == True].feature_name.describe())
# Min / Max 'subscription_date':
print("\n\nChurned users subscription_date: min - %s, max - %s" % 
      (data_df[data_df['renewed'] == False].subscription_date.min(),
       data_df[data_df['renewed'] == False].subscription_date.max()))
print("Renewed users subscription_date: min - %s, max - %s" % 
      (data_df[data_df['renewed'] == True].subscription_date.min(),
       data_df[data_df['renewed'] == True].subscription_date.max()))
# Min / Max 'redshifted_at':
print("\nChurned users redshifted_at: min - %s, max - %s" % 
      (data_df[data_df['renewed'] == False].redshifted_at.min(),
       data_df[data_df['renewed'] == False].redshifted_at.max()))
print("Renewed users redshifted_at: min - %s, max - %s" % 
      (data_df[data_df['renewed'] == True].redshifted_at.min(),
       data_df[data_df['renewed'] == True].redshifted_at.max()))



Data head:
     id  renewed   subscription_date  feature_name       redshifted_at
0  5147    False 2017-10-13 17:12:41            20 2017-10-21 13:05:45
1  7423    False 2017-10-08 19:18:56            25 2017-10-08 19:52:00
2  3491     True 2017-10-10 15:08:54            18 2017-10-11 16:47:53
3  9891     True 2017-10-15 01:12:10             3 2017-10-15 01:22:11
4  6378     True 2017-10-23 00:31:45            35 2017-10-28 16:57:53

Data shape: (1162830, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162830 entries, 0 to 1162829
Data columns (total 5 columns):
id                   1162830 non-null int64
renewed              1162830 non-null bool
subscription_date    1162830 non-null datetime64[ns]
feature_name         1162830 non-null int64
redshifted_at        1162830 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](2), int64(2)
memory usage: 36.6 MB

Data info: 
None

Missing values: 
id                   0
renewed              0
subscription_date    0
feature_name   

In [124]:
# TODO: REMOVE load and datetime conv!!
# Load the data
data_df = pd.read_csv(dataFilePath)

# Convert relevant columns to datetime type:
date_cols = ['subscription_date', 'redshifted_at']
for col in date_cols:
    data_df.loc[:, col] = pd.to_datetime(data_df.loc[:, col])

In [125]:
# # # # DATA PREPROCESSING (*feature extraction, feature reduction and feature scaling) # # # # 

# Label extraction - 'churned' (!renewed):
'''(apparently ~data_df.loc[:, 'renewed'] returns -1 / -2 and not True / False)'''
data_df[label_col_name] = (data_df['renewed'] == False)


# FEATURE EXTRACTION
# Compute days since subsrciption (redshifted_at - subscription_date):
data_df['days_since_sub'] = (data_df['redshifted_at'] - data_df['subscription_date']).dt.ceil("D").dt.days
# Compute weeks since subscription (w1:0-7, w2:8-14, w3:15-21, w4:22-28)
data_df['weeks_since_sub'] = pd.to_numeric(np.ceil(data_df['days_since_sub'] / 7), downcast='signed')
''' I think the feature_name that was used during subscription could be a very interesting feature 
but unfortunately there are only 69 such instances and all (but 1) have less than 10 occurrences, 
so we better just add them to week 1. '''
data_df.loc[data_df.loc[:, 'weeks_since_sub'] == 0, 'weeks_since_sub'] = 1

# Generate features - combinations of feature_name & weeks from subscription 
# (e.g. w2_f1 represents the usage of feature_name 1 during week 2)
groupby_cols = ['id', label_col_name, 'feature_name', 'weeks_since_sub'] 
user_fe_act_per_week_df = data_df.groupby(groupby_cols).size().unstack(['weeks_since_sub', 'feature_name'])
new_col_names = ['w' + str(a) + '_f' + str(b)  for a,b in user_fe_act_per_week_df.columns.get_values()] 
user_fe_act_per_week_df.columns = new_col_names
user_fe_act_per_week_df.reset_index()

# Preprocessed data - we use deep copy in case we would like to extract additional features from user_fe_act_per_week_df, 
#                     e.g. # of unique feature_names used in week
pp_data_df = user_fe_act_per_week_df.copy(deep=True) 

DEBUG: preprocessed data (pp_data_df) shape BEFORE: (10167, 256)
DEBUG: Combining 31 features into 4 new features
DEBUG: preprocessed data (pp_data_df) shape AFTER: (10167, 229)


In [122]:
# # # # DATA PREPROCESSING (feature extraction, *feature reduction and feature scaling) # # # # 

# FEATURE REDUCTION
# Combining rare features per week (w1_other, w2_other, ...):
fe_missing_val_sr = user_fe_act_per_week_df.isnull().sum( )
num_of_users = user_fe_act_per_week_df.shape[0]
missing_val_threshold = 0.999 # rare feature - less than 0.1% occurences 
rare_fe_list = fe_missing_val_sr[(fe_missing_val_sr / num_of_users) > missing_val_threshold].index

pp_data_df.fillna(0, inplace=True)

print("DEBUG: preprocessed data (pp_data_df) shape BEFORE: " + str(pp_data_df.shape))
for i in range(1, 5):
    w_rare_fe_list = rare_fe_list[np.where([x.startswith('w' + str(i) + '_') for x in rare_fe_list])[0]] # finds the rare features of the relevant week
    w_rare_fe_list
    pp_data_df['w' + str(i) + '_other'] = pp_data_df[w_rare_fe_list].sum(axis=1)
print("Combining %d features into %d new features" % (len(rare_fe_list), i))
pp_data_df.drop(columns=rare_fe_list, inplace=True)
print("DEBUG: preprocessed data (pp_data_df) shape AFTER: " + str(pp_data_df.shape))
# DEBUG: print('w1_f22' in pp_data_df.columns[np.where([x.startswith('w1_') for x in pp_data_df.columns])[0]]) # should be False


DEBUG: preprocessed data (pp_data_df) shape BEFORE: (10167, 256)


In [ ]:
# # # # DATA PREPROCESSING (feature extraction, feature reduction and *feature scaling) # # # # 

# FEATURE SCALING
# Apply log-transformation to all features:
pp_data_df = pp_data_df.apply(lambda x: np.log(x + 1))




In [108]:
# TODO: this should be done once we have the training dataset (after the split!)
# REMOVE OUTLIERS 
from sklearn.neighbors import LocalOutlierFactor

# Fit LOF
lof_clf = LocalOutlierFactor(n_neighbors=20, n_jobs=-1)
lof_pred = lof_clf.fit_predict(design_mat_df.drop(columns=labelColName))
print(("DEBUG: Before outliers cleaning - pct of class 1 samples = %.4f") % 
      (100*sum(pp_data_df[labelColName])/pp_data_df.shape[0]))
outlier_idx = np.where(lof_pred == -1)[0]
pp_data_df.drop(index=outlier_idx, inplace=True)
print(("DEBUG: After outliers cleaning - pct of class 1 samples = %.4f") % 
      (100*sum(pp_data_df[labelColName])/pp_data_df.shape[0]))
print(("%d outlier samples were deleted.") % len(outlier_idx))

#data_df[data_df['renewed'] == True].weeks_since_sub.value_counts(normalize=True)
#data_df[data_df['renewed'] == False].weeks_since_sub.value_counts(normalize=True)

#data_df[data_df['days_since_sub'] == 0].feature_name.value_counts()



,,w1_f3,w1_f15,w1_f16,w1_f31,w2_f31,w1_f32,w2_f32,w1_f6,w1_f7,w2_f14,...,w3_f36,w3_f41,w3_f51,w3_f55,w4_f42,w2_f42,w1_other,w2_other,w3_other,w4_other
id,churned,,,,,,,,,,,,,,,,,,,,,
1,False,1.0000,3.0000,1.0000,1.0000,1.0000,5.0000,5.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,False,2.0000,0.0000,0.0000,6.0000,0.0000,4.0000,0.0000,3.0000,5.0000,1.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,True,1.0000,4.0000,1.0000,2.0000,0.0000,5.0000,0.0000,1.0000,2.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,False,0.0000,0.0000,20.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,True,4.0000,2.0000,1.0000,5.0000,0.0000,2.0000,0.0000,5.0000,5.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [117]:
#pp_data_df.astype(bool).sum(axis=0)#[w_rare_fe_list]
#'churned' in user_fe_act_per_week_df.columns 

pp_data_df.describe()

,w1_f3,w1_f15,w1_f16,w1_f31,w2_f31,w1_f32,w2_f32,w1_f6,w1_f7,w2_f14,...,w3_f36,w3_f41,w3_f51,w3_f55,w4_f42,w2_f42,w1_other,w2_other,w3_other,w4_other
count,"10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000",...,"10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000","10,167.0000"
mean,0.9869,1.1640,0.8103,1.0547,0.4224,0.8533,0.2933,0.5520,0.6367,0.2029,...,0.0011,0.0010,0.0017,0.0016,0.0009,0.0017,0.0015,0.0022,0.0024,0.0027
std,0.9123,1.0950,0.8999,0.9068,0.7364,0.8780,0.6241,0.6392,0.7131,0.5297,...,0.0331,0.0275,0.0377,0.0347,0.0253,0.0343,0.0388,0.0533,0.0540,0.0561
min,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,0.6931,1.0986,0.6931,1.0986,0.0000,0.6931,0.0000,0.6931,0.6931,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
75%,1.6094,1.9459,1.3863,1.6094,0.6931,1.3863,0.0000,1.0986,1.0986,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
max,5.1533,5.6204,5.7301,5.3033,4.7622,5.6937,5.2730,4.3820,3.9318,4.2195,...,1.7918,1.0986,1.7918,1.0986,1.0986,0.6931,1.6094,2.1972,1.7918,2.3026


In [114]:
np.log(6)

1.791759469228055

In [ ]:
len(data_df[data_df['renewed'] == False].id.value_counts()) # number of churned users
#data_df[data_df['renewed'] == True].groupby('id').size().reset_index(name='counts').shape
#sum(data_df.subscription_date <= data_df.redshifted_at)

In [ ]:
data_df.shape

In [ ]:
8012 / 10167


In [ ]:
2155/10167

In [ ]:
2155+8012

In [20]:
data_df[label_col_name] = (data_df['renewed'] == False)


In [28]:
data_df.shape

(1162830, 8)

In [29]:
user_fe_act_per_week_df.shape

(10167, 256)

In [30]:
data_df[data_df['id']==1].shape

(17, 8)

In [31]:
data_df[data_df['id']==1].renewed.sum()

17

In [32]:
(data_df[data_df['id']==1].churned == False).sum()

17

In [33]:
data_df[data_df['id']==1]

,id,renewed,subscription_date,feature_name,redshifted_at,churned,days_since_sub,weeks_since_sub
211460,1,True,2017-10-01 00:01:50,32,2017-10-11 12:33:04,False,11,2
212309,1,True,2017-10-01 00:01:50,32,2017-10-05 21:14:25,False,5,1
215229,1,True,2017-10-01 00:01:50,16,2017-10-01 00:07:24,False,1,1
220296,1,True,2017-10-01 00:01:50,15,2017-10-01 00:03:07,False,1,1
223982,1,True,2017-10-01 00:01:50,32,2017-10-06 17:33:48,False,6,1
229741,1,True,2017-10-01 00:01:50,32,2017-10-05 20:33:10,False,5,1
241257,1,True,2017-10-01 00:01:50,31,2017-10-05 21:15:56,False,5,1
243544,1,True,2017-10-01 00:01:50,32,2017-10-05 21:17:09,False,5,1
252264,1,True,2017-10-01 00:01:50,31,2017-10-13 03:45:24,False,13,2
262221,1,True,2017-10-01 00:01:50,15,2017-10-01 00:05:47,False,1,1


In [34]:
data_df

,id,renewed,subscription_date,feature_name,redshifted_at,churned,days_since_sub,weeks_since_sub
0,5147,False,2017-10-13 17:12:41,20,2017-10-21 13:05:45,True,8,2
1,7423,False,2017-10-08 19:18:56,25,2017-10-08 19:52:00,True,1,1
2,3491,True,2017-10-10 15:08:54,18,2017-10-11 16:47:53,False,2,1
3,9891,True,2017-10-15 01:12:10,3,2017-10-15 01:22:11,False,1,1
4,6378,True,2017-10-23 00:31:45,35,2017-10-28 16:57:53,False,6,1
5,2873,True,2017-10-28 02:47:51,16,2017-10-28 02:57:06,False,1,1
6,1119,True,2017-10-07 20:17:37,62,2017-10-08 20:21:59,False,2,1
7,5751,True,2017-10-29 14:41:33,14,2017-10-30 19:45:21,False,2,1
8,6875,True,2017-10-18 16:51:36,2,2017-10-22 20:20:42,False,5,1
9,721,True,2017-10-08 14:38:52,26,2017-10-16 18:24:00,False,9,2
